<a href="https://colab.research.google.com/github/sambhavnoobcoder/Lecture-Notes-Chatbot/blob/main/Lecture_Notes_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
!pip install google-generativeai requests faiss-cpu sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/

## Initialise the libraries and the model parameters




lets first initilaise the libraries




In [2]:
import google.generativeai as genai
import requests
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from bs4 import BeautifulSoup
import gradio as gr

initialising model parameters

In [3]:
# Configure Gemini API key
gemini_api_secret_name = 'GEMINI'

from google.colab import userdata

try:
    GOOGLE_API_KEY = userdata.get(gemini_api_secret_name)
    genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
    print(f'Secret not found\n\nThis expects you to create a secret named {gemini_api_secret_name} in Colab\n\nVisit https://makersuite.google.com/app/apikey to create an API key\n\nStore that in the secrets section on the left side of the notebook (key icon)\n\nName the secret {gemini_api_secret_name}')
    raise e
except userdata.NotebookAccessError as e:
    print(f'You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.')
    raise e
except Exception as e:
    # unknown error
    print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
    raise e

# DATA EXTRACTION

lets start by coding up the helper functions tp fetch the lecture notes and model architectures.

In [4]:
# Fetch lecture notes and model architectures
def fetch_lecture_notes():
    lecture_urls = [
        "https://stanford-cs324.github.io/winter2022/lectures/introduction/",
        "https://stanford-cs324.github.io/winter2022/lectures/capabilities/",
        "https://stanford-cs324.github.io/winter2022/lectures/data/",
        "https://stanford-cs324.github.io/winter2022/lectures/modeling/"
    ]
    lecture_texts = []
    for url in lecture_urls:
        response = requests.get(url)
        if response.status_code == 200:
            print(f"Fetched content from {url}")
            lecture_texts.append((extract_text_from_html(response.text), url))
        else:
            print(f"Failed to fetch content from {url}, status code: {response.status_code}")
    return lecture_texts

def fetch_model_architectures():
    url = "https://github.com/Hannibal046/Awesome-LLM#milestone-papers"
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Fetched model architectures, status code: {response.status_code}")
        return extract_text_from_html(response.text), url
    else:
        print(f"Failed to fetch model architectures, status code: {response.status_code}")
        return "", url

In [5]:
# Extract text from HTML content
def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    for script in soup(["script", "style"]):
        script.extract()
    text = soup.get_text(separator="\n", strip=True)
    return text

# Data Preprocessing

Lets start with creating the embeddings for this project . use sentenceTransformer from Transformer libraray for the same

In [6]:
# Generate embeddings using SentenceTransformers
def create_embeddings(texts, model):
    texts_only = [text for text, _ in texts]
    embeddings = model.encode(texts_only)
    return embeddings

Lets add some Opensource vector indexing , such as by using FAISS .
Source : [faiss github](https://github.com/facebookresearch/faiss)

In [7]:
# Initialize FAISS index
def initialize_faiss_index(embeddings):
    dimension = embeddings.shape[1]  # Assuming all embeddings have the same dimension
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings.astype('float32'))
    return index

# Querying Logic

Handle logic for querying

In [8]:
# Handle natural language queries
conversation_history = []

def handle_query(query, faiss_index, embeddings_texts, model):
    global conversation_history

    query_embedding = model.encode([query]).astype('float32')

    # Search FAISS index
    _, indices = faiss_index.search(query_embedding, 3)  # Retrieve top 3 results
    relevant_texts = [embeddings_texts[idx] for idx in indices[0]]

    # Combine relevant texts and truncate if necessary
    combined_text = "\n".join([text for text, _ in relevant_texts])
    max_length = 500  # Adjust as necessary
    if len(combined_text) > max_length:
        combined_text = combined_text[:max_length] + "..."

    # Generate a response using Gemini
    try:
        response = genai.generate_text(
            model="models/text-bison-001",
            prompt=f"Based on the following context:\n\n{combined_text}\n\nAnswer the following question: {query}",
            max_output_tokens=200
        )
        generated_text = response.result
    except Exception as e:
        print(f"Error generating text: {e}")
        generated_text = "An error occurred while generating the response."

    # Update conversation history
    conversation_history.append(f"User: {query}")
    conversation_history.append(f"System: {generated_text}")

    # Extract sources
    sources = [url for _, url in relevant_texts]

    return generated_text, sources

add function for summaraisation

In [9]:
def generate_concise_response(prompt, context):
    try:
        response = genai.generate_text(
            model="models/text-bison-001",
            prompt=f"{prompt}\n\nContext: {context}\n\nAnswer:",
            max_output_tokens=200
        )
        return response.result
    except Exception as e:
        print(f"Error generating concise response: {e}")
        return "An error occurred while generating the concise response."

# Gradio Chatbot Interface

In [10]:
# Main function to execute the pipeline
def chatbot(message , history):
    lecture_notes = fetch_lecture_notes()
    model_architectures = fetch_model_architectures()

    all_texts = lecture_notes + [model_architectures]

    # Load the SentenceTransformers model
    embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    embeddings = create_embeddings(all_texts, embedding_model)

    # Initialize FAISS index
    faiss_index = initialize_faiss_index(np.array(embeddings))


    response, sources = handle_query(message, faiss_index, all_texts, embedding_model)
    print("Query:", message)
    print("Response:", response)
    total_text = response
    if sources:
        print("Sources:", sources)
        relevant_source = ""
        for source in sources:
          relevant_source += source +"\n"
        total_text += "\n\nSources:\n" + relevant_source

    else:
        print("Sources: None of the provided sources were used.")
    print("----")

    # Generate a concise and relevant summary using Gemini
    prompt = "Summarize the user queries so far"
    user_queries_summary = " ".join(message)
    concise_response = generate_concise_response(prompt, user_queries_summary)
    print("Concise Response:")
    print(concise_response)
    return total_text

iface = gr.ChatInterface(
    chatbot,
    title="LLM Research Assistant",
    description="Ask questions about LLM architectures, datasets, and training techniques.",
    examples=[
        "What are some milestone model architectures in LLMs?",
        "Explain the transformer architecture.",
        "Tell me about datasets used to train LLMs.",
        "How are LLM training datasets cleaned and preprocessed?",
        "Summarize the user queries so far"
    ],
    retry_btn="Regenerate",
    undo_btn="Undo",
    clear_btn="Clear",
)

# Main function Calling

In [11]:
if __name__ == "__main__":
    iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9ad14816e72c3d89cc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9ad14816e72c3d89cc.gradio.live
